# Age, Cholesterol, Blood Pressure, and Chest Pain in Determining Heart Disease

Andy Hong, Celine Li, Jason Shi, Ibrahim Saeed

# Preliminary Data Analysis

In [1]:
# Here we load the packages necessary to run our functions
library(tidyverse)
library(repr)
library(tidymodels)

options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

Preliminary Exploratory Data Analysis

In the code below, we can prove that the dataset can be read from the web. We will be naming the dataset "heart_data". 


In [2]:
heart_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", 
                       col_names = FALSE)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


We notice that there are no column names for the data set thus we will need to add column names based on what each column represents.

In [3]:
names <- c("age","sex","cp","trestbps",
           "chol","fbs","restecg","thalach",
           "exang","oldpeak","slope",
           "ca","thal","num")

colnames(heart_data) <- names
heart_data

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1
38,1,3,138,175,0,0,173,0,0.0,1,?,3.0,0


Let's select for the columns we are interested in and tidy it up. Note that the "num" column tells us the prescence of heart disease within a patient, with a value of 0 meaning no heart disease present and a value of 1 meaning heart disease is present with greater values representing how significant the prescence is all the way up to the maximum value of 4. 

In [4]:
heart_data_select <- select(heart_data, age, cp, trestbps, chol, num)
slice(heart_data_select,1:5)


age,cp,trestbps,chol,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
63,1,145,233,0
67,4,160,286,2
67,4,120,229,1
37,3,130,250,0
41,2,130,204,0


Each row is one unique obsevation and each cell only contains one value. But we are only interested in whether a patient has heart diesease or not. Let's transform the "num" column so that values equal to 0 mean no heart diesease is present whereas any values higher means heart diesease is present. 

In [5]:
heart_clean <- heart_data_select |>
               mutate(diag = ifelse(is.na(num), NA, (num > 0))) 

Now let's check to see if any missing values. To do this, we can inbed the is.na() function into the sum() function. 

In [6]:
sum(is.na(heart_data_select))

[1] 0

The value 0 tells us that there are no missing values in our data frame. 

Let's separate our training data from the testing data, set a seed, and summarize the training idea to get a good idea of what we are working with.

In [7]:
#splitting dataframe into training, testing datasets
set.seed(1)

heart_split <- initial_split(heart_clean, prop = 3/4, strata = num)
heart_training <- training(heart_split)
heart_testing <- testing(heart_split)


We then can summarize this data into a table below.

In [8]:
#summarizing training data into a table and getting mean, min, max
heart_summarized <- heart_training %>%
group_by(diag) %>%
summarize(min_age = min(age), max_age = max(age), mean_age = mean(age),
          min_cp  = min(cp), max_cp = max(cp), mean_cp = mean(cp),
          min_chol = min(chol), max_chol = max(chol), mean_chol = mean(chol),
          min_trestbps = min(trestbps), max_trestbps = max(trestbps), 
                             mean_trestbps = mean(trestbps), no_patients = n())
heart_summarized
          
          


diag,min_age,max_age,mean_age,min_cp,max_cp,mean_cp,min_chol,max_chol,mean_chol,min_trestbps,max_trestbps,mean_trestbps,no_patients
<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
FALSE,29,76,52.68548,1,4,2.774194,126,564,244.2903,94,180,129.7339,124
TRUE,35,77,56.48039,1,4,3.578431,131,409,249.3627,100,192,133.9020,102


We see that there are no anomalies and the number of observations with respect to the diagnosis value (whether or not the patent has heart diesease) is fair. "no_patients" tells us the number of rows. 

In [5]:
# #Plotting this data to find correlation of diagnosis
# options(repr.plot.width = 8, repr.plot.height = 7)

# # RColorBrewer features for color blindness 
# library(RColorBrewer)


# graph <- ggplot(heart_training, aes(x = age, fill = diag, color = diag)) +
#                  geom_histogram() +
#                  labs(x = "Age (years)", y = "Number of patients", color = "Diagnosis", fill = "Diagnosis", title = "Distribution of Age") +
#                  theme(text = element_text(size = 15)) +
#                  scale_fill_brewer(palette = "Dark2") 

# graph2 <- ggplot(heart_training, aes(x = cp, color = diag, fill = diag)) +
#                  geom_histogram(bins = 15) +
#                  labs(x = "Chest Pain", y = "Number of patients", color = "Diagnosis", fill = "Diagnosis", title = "Distribution of Chest Pains") +
#                  theme(text = element_text(size = 15))+
#                  scale_fill_brewer(palette = "Dark2") 

# graph3 <- ggplot(heart_training, aes(x = trestbps, color = diag, fill = diag)) +
#                  geom_histogram() +
#                  labs(x = "Age (years)", y = "Number of patients",
#                       color = "Diagnosis", fill = "Diagnosis", title = "Distribution of Resting Blood Pressure") +
#                  theme(text = element_text(size = 15))+
#                  scale_fill_brewer(palette = "Dark2") 

# graph4 <- ggplot(heart_training, aes(x = chol,  color = diag, fill = diag)) +
#                  geom_histogram() +
#                  labs(x = "Cholesterol (mg/dl))", y = "Number of patients", 
#                       color = "Diagnosis", fill = "Diagnosis", title = "Distribution of Cholesterol levels") +
#                  theme(text = element_text(size = 15))+
#                  scale_fill_brewer(palette = "Dark2") 

# graph
# graph2
# graph3
# graph4

-notes to self for this final project

-use a func to align the histograms side by side; there was a func to do this but i forgot but go check 
-use the foward output algorithm to find the num of best predictor variables in ch.6.8 within the textbook
-the table was supposedly good but i will have to start working alot on wednesday to see if everything is good on thurs
-also they open on tues so can also go tuesday start 3:30 to ask tony questions 
-holyfuck doomed gg 

## Predictor Variable Selection Analysis 